## 情感分析项目

本项目的目标是基于用户提供的评论，通过算法自动去判断其评论是正面的还是负面的情感。比如给定一个用户的评论：
- 评论1： “我特别喜欢这个电器，我已经用了3个月，一点问题都没有！”
- 评论2： “我从这家淘宝店卖的东西不到一周就开始坏掉了，强烈建议不要买，真实浪费钱”

对于这两个评论，第一个明显是正面的，第二个是负面的。 我们希望搭建一个AI算法能够自动帮我们识别出评论是正面还是负面。

情感分析的应用场景非常丰富，也是NLP技术在不同场景中落地的典范。比如对于一个证券领域，作为股民，其实比较关注舆论的变化，这个时候如果能有一个AI算法自动给网络上的舆论做正负面判断，然后把所有相关的结论再整合，这样我们可以根据这些大众的舆论，辅助做买卖的决策。 另外，在电商领域评论无处不在，而且评论已经成为影响用户购买决策的非常重要的因素，所以如果AI系统能够自动分析其情感，则后续可以做很多有意思的应用。 

情感分析是文本处理领域经典的问题。整个系统一般会包括几个模块：
- 数据的抓取： 通过爬虫的技术去网络抓取相关文本数据
- 数据的清洗/预处理：在本文中一般需要去掉无用的信息，比如各种标签（HTML标签），标点符号，停用词等等
- 把文本信息转换成向量： 这也成为特征工程，文本本身是不能作为模型的输入，只有数字（比如向量）才能成为模型的输入。所以进入模型之前，任何的信号都需要转换成模型可识别的数字信号（数字，向量，矩阵，张量...)
- 选择合适的模型以及合适的评估方法。 对于情感分析来说，这是二分类问题（或者三分类：正面，负面，中性），所以需要采用分类算法比如逻辑回归，朴素贝叶斯，神经网络，SVM等等。另外，我们需要选择合适的评估方法，比如对于一个应用，我们是关注准确率呢，还是关注召回率呢？ 

在本次项目中，我们已经给定了训练数据和测试数据，它们分别是 ``train.positive.txt``, ``train.negative.txt``， ``test_combined.txt``. 请注意训练数据和测试数据的格式不一样，详情请见文件内容。 整个项目你需要完成以下步骤：

数据的读取以及清洗： 从给定的.txt中读取内容，并做一些数据清洗，这里需要做几个工作： 
- （1） 文本的读取，需要把字符串内容读进来。 
- （2）去掉无用的字符比如标点符号，多余的空格，换行符等 
- （3） 把文本转换成``TF-IDF``向量： 这部分直接可以利用sklearn提供的``TfidfVectorizer``类来做。
- （4） 利用逻辑回归等模型来做分类，并通过交叉验证选择最合适的超参数

项目中需要用到的数据：
- ``train.positive.txt``, ``train.negative.txt``， ``test_combined.txt``： 训练和测试数据
- ``stopwords.txt``： 停用词库


你需要完成的部分为标记为`TODO`的部分。 

另外，提交作业时候的注意点：
> 1. 不要试图去创建另外一个.ipynb文件，所有的程序需要在`starter_code.ipynb`里面实现。很多的模块已经帮你写好，不要试图去修改已经定义好的函数以及名字。 当然，自己可以按需求来创建新的函数。但一定要按照给定的框架来写程序，不然判作业的时候会出现很多问题。 
> 2. 上传作业的时候把整个文件解压成.zip文件（不要.rar格式），请不要上传图片文件，其他的都需要上传包括`README.md`。
> 3. 确保程序能够正常运行，我们支持的环境是`Python 3`,  千万不要使用`Python 2`
> 4. 上传前一定要确保完整性，批改过一次的作业我们不会再重新批改，会作为最终的分数来对待。 
> 5. 作业可以讨论，但请自己完成。让我们一起遵守贪心学院的`honor code`。

### 1. File Reading: 文本读取 

In [260]:
import pandas as pd
import numpy as np
import os
from bs4 import BeautifulSoup

In [261]:
def read_txt_comments(file_name, label = False):
    file = open(file_name, mode = 'r')
    lines = file.read()
    file.close()
    # use bs4 parse data
    soup = BeautifulSoup(lines, 'html.parser')
    review = soup.findAll('review')
    review_list = []
    if label == True:
        label_list = []
    for i in range(len(review)):
        a_review = review[i].get_text().lstrip().rstrip().replace('\n', ' ')
        review_list.append(a_review)
        if label == True:
            a_label = review[i].get('label')
            label_list.append(a_label)
    if label == False:        
        return review_list
    else:
        return review_list, label_list

In [262]:
def process_file(train_pos_file, train_neg_file, test_comb_file):  
    # TODO: 读取文件部分，把具体的内容写入到变量里面
    train_pos_comments = read_txt_comments(train_pos_file)
    train_neg_comments = read_txt_comments(train_neg_file)
    train_comments = train_pos_comments + train_neg_comments
    train_labels = [1]*len(train_pos_comments) + [0]*len(train_neg_comments)
    test_comments, test_labels = read_txt_comments(test_comb_file, label = True)
    return train_pos_comments, train_neg_comments, train_comments, train_labels, test_comments, test_labels

In [263]:
train_pos_file = "train_positive.txt"
train_neg_file = "train_negative.txt"
test_comb_file = "test_combined.txt"

In [264]:
train_pos_comments, train_neg_comments, train_comments, train_labels, test_comments, test_labels = process_file(train_pos_file, train_neg_file, test_comb_file)

### 2. Explorary Analysis: 做一些简单的可视化分析

In [265]:
# 训练数据和测试数据大小
print(len(train_comments), len(test_comments))

8065 2500


> 这里有一个假设想验证。我觉得，如果一个评论是负面的，则用户留言时可能会长一些，因为对于负面的评论，用户很可能会把一些细节写得很清楚。但对于正面的评论，用户可能就只写“非常好”，这样的短句。我们想验证这个假设。 为了验证这个假设，打算画两个直方图，分别对正面的评论和负面的评论。 具体的做法是：1. 把正面和负面评论分别收集，之后分别对正面和负面评论画一个直方图。 2.  直方图的X轴是评论的长度，所以从是小到大的顺序。然后Y轴是对于每一个长度，出现了多少个正面或者负面的评论。 通过两个直方图的比较，即可以看出``评论``是否是一个靠谱的特征。


In [266]:
# TODO: 对于训练数据中的正负样本，分别画出一个histogram， histogram的x抽是每一个样本中字符串的长度，y轴是拥有这个长度的样本的百分比。
#       并说出样本长度是否对情感有相关性 (需要先用到结巴分词)
#       参考：https://baike.baidu.com/item/%E7%9B%B4%E6%96%B9%E5%9B%BE/1103834?fr=aladdin
#       画饼状图参考： https://pythonspot.com/matplotlib-histogram/   
#                   https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.hist.html
#!pip3 install jieba
import jieba

In [267]:
import plotly.graph_objects as go
import numpy as np

pos = [len(jieba.lcut(x)) for x in train_pos_comments]
neg = [len(jieba.lcut(x)) for x in train_neg_comments]

fig = go.Figure()
fig.add_trace(go.Histogram(x = pos, histnorm='probability', name = 'pos'))
fig.add_trace(go.Histogram(x = neg, histnorm='probability', name = 'neg'))

# Overlay both histograms
fig.update_layout(barmode='overlay')
# Reduce opacity to see both histograms
fig.update_traces(opacity=0.75)
fig.show()

In [268]:
from scipy.stats import ks_2samp
ks_2samp(pos, neg)

Ks_2sampResult(statistic=0.0975399673735726, pvalue=1.0)

// TODO: 情感跟评论长度是否有相关性？

// From the histogram, positive comments have a larger proportion of "<25 words comments", compared with the negative comments. But from the KS test result, the difference is not significant. 


### 3. 文本预处理
> 在此部分需要做文本预处理方面的工作。 分为几大块：
- ``去掉特殊符号``  比如#$.... 这部分的代码已经给出，不需要自己写
- ``把数字转换成特殊单词`` 把数字转换成 " NUM "， 这部分需要写。 注意：NUM前面和后面加一个空格，这样可以保证之后分词时被分掉。
- ``分词并过滤掉停用词`` 停用词库已经提供，需要读取停用词库，并按照此停用词库做过滤。 停用词库使用给定的文件：``stopwords.txt`` 

In [269]:
# load stopwords.txt
file = open('stopwords.txt', mode = 'r')
stop_words = file.read()
file.close()

stop_words = list(stop_words)

In [270]:
import re
def clean_nums_symbols(text):
    """
    对特殊符号做一些处理，此部分已写好。如果不满意也可以自行改写，不记录分数。
    """
    text = re.sub(r'\d+[\d.]*', 'NUM', text)
    text = re.sub('[!！]+', "!", text)
    text = re.sub('[?？]+', "?", text)
    text = re.sub("[a-zA-Z#$%&\'()*+,-./:;：<=>@，。★、…【】《》“”‘’[\\]^_`{|}~～]+ó", " OOV ", text)
    return re.sub("\s+", " ", text)  

def clean_nums_symbols_list(comments_list):
    clean_comments_list = []
    for text in comments_list:
        clean_comments_list.append(clean_nums_symbols(text))
    return clean_comments_list

def remove_stop_words(clean_comments_list):
    list_tokens = [jieba.lcut(x) for x in clean_nums_symbols_list(clean_comments_list)]
    clean_list_tokens = []
    for tokens in list_tokens:    
        clean_tokens = tokens[:]
        for token in tokens:
            if token in stop_words:
                clean_tokens.remove(token)
        clean_list_tokens.append(clean_tokens)
    return clean_list_tokens

# TODO：对于train_comments, test_comments进行字符串的处理，几个考虑的点：
#   1. 去掉特殊符号
#   2. 把数字转换成特殊字符或者单词
#   3. 分词并做停用词过滤
#   4. ... （或者其他）
#
#   需要注意的点是，由于评论数据本身很短，如果去掉的太多，很可能字符串长度变成0
#   预处理部分，可以自行选择合适的方.

clean_train_comments = clean_nums_symbols_list(train_comments)
clean_train_token = remove_stop_words(clean_train_comments)

clean_test_comments = clean_nums_symbols_list(test_comments)
clean_test_token = remove_stop_words(clean_test_comments)

In [271]:
train_comments_cleaned = [' '.join(x) for x in clean_train_token]
test_comments_cleaned = [' '.join(x) for x in clean_test_token]

In [272]:
# 打印一下看看
print(train_comments_cleaned[0], test_comments_cleaned[0])
print(train_comments_cleaned[1], test_comments_cleaned[1])

请问 这机 不是 有个 遥控器 终于 找到 同道中人 初中 开始 已经 喜欢 michaeljackson 同学 都 鄙夷 眼光 他们 人为 jackson 样子 古怪 甚至 ＂ 丑 ＂ 当场 气晕 现在 同道中人 好开心 ! michaeljacksonisthemostsuccessfulsingerintheworld !
发短信 特别 方便 ! 背后 屏幕 很大 起来 舒服 手触 屏 ! 切换 屏幕 很 麻烦 ! 完 深夜 两点 却 坐在 电脑前 情难 自禁 这是 最好 结局 惟有 如此 就让 前世 今生 纠结 停留 此刻 相逢 人生 不再 唏嘘 他们 身心 居 一处 可是 还是 痛心 这样 这样 爱


### 4. 把文本转换成向量
> 预处理好文本之后，我们就需要把它转换成向量形式，这里我们使用tf-idf的方法。 sklearn自带此功能，直接调用即可。输入就是若干个文本，输出就是每个文本的tf-idf向量。详细的使用说明可以在这里找到： 参考：https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html  这里需要特别注意的一点是：对于训练数据调用fit_transform, 也就是训练的过程。 但对于测试数据，不能再做训练，而是直接使用已经训练好的object做transform操作。思考一下为什么要这么做？


In [273]:
X_train = vectorizer.fit_transform(train_corpus)
X_test = vectorizer.transform(test_corpus)
print(X_train.shape)
print(X_test.shape)

(8065, 26675)
(2500, 26675)


In [274]:
# TODO: 利用tf-idf从文本中提取特征,写到数组里面. 
#       参考：https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
from sklearn.feature_extraction.text import TfidfVectorizer
train_corpus = train_comments_cleaned
test_corpus = test_comments_cleaned
vectorizer = TfidfVectorizer()

X_train =  vectorizer.fit_transform(train_corpus)
y_train =  train_labels
X_test =   vectorizer.transform(test_corpus)
y_test =   test_labels

print(np.shape(X_train), np.shape(X_test), np.shape(y_train), np.shape(y_test))

(8065, 26675) (2500, 26675) (8065,) (2500,)


### 5. 通过交叉验证来训练模型
> 接下来需要建模了！ 这里我们分别使用逻辑回归，朴素贝叶斯和SVM来训练。针对于每一个方法我们使用交叉验证（gridsearchCV)， 并选出最好的参数组合，然后最后在测试数据上做验证。 这部分已经在第二次作业中讲过。

In [302]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

# TODO： 利用逻辑回归来训练模型
#       1. 评估方式： F1-score
#       2. 超参数（hyperparater）的选择利用grid search https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html
#       3. 打印出在测试数据中的最好的结果（precision, recall, f1-score, 需要分别打印出正负样本，以及综合的）
#       请注意：做交叉验证时绝对不能用测试数据。 测试数据只能用来最后的”一次性“检验。
#       逻辑回归的使用方法请参考：http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
#       对于逻辑回归，经常调整的超参数为： C
params_c = np.logspace(-5,2,15)
parameters = {'C': params_c}
clf = LogisticRegression(random_state=0)
model = GridSearchCV(clf, parameters, scoring = 'f1', cv = 5)
model.fit(X_train, y_train)

# print the best parameter
print(model.best_params_)

# print confusion matrix
predictions = model.predict(X_test)
print(classification_report(y_test, predictions.astype(str)))

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning:

Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.



{'C': 1.0}
              precision    recall  f1-score   support

           0       0.87      0.55      0.67      1250
           1       0.67      0.92      0.77      1250

    accuracy                           0.73      2500
   macro avg       0.77      0.73      0.72      2500
weighted avg       0.77      0.73      0.72      2500



In [305]:
from sklearn.naive_bayes import MultinomialNB
# TODO： 利用朴素贝叶斯来训练模型
#       1. 评估方式： F1-score
#       2. 超参数（hyperparater）的选择利用grid search https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html
#       3. 打印出在测试数据中的最好的结果（precision, recall, f1-score, 需要分别打印出正负样本，以及综合的）
#       请注意：做交叉验证时绝对不能用测试数据。 测试数据只能用来最后的”一次性“检验。
#       朴素贝叶斯的使用方法请参考：https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html#sklearn.naive_bayes.MultinomialNB
#       对于朴素贝叶斯，一般不太需要超参数的调节。但如果想调参，也可以参考上面的链接，有几个参数是可以调节的。 

clf = MultinomialNB()
clf.fit(X_train, y_train)

# print confusion matrix
predictions = clf.predict(X_test)
print(classification_report(y_test, predictions.astype(str)))


              precision    recall  f1-score   support

           0       0.94      0.31      0.47      1250
           1       0.59      0.98      0.73      1250

    accuracy                           0.65      2500
   macro avg       0.76      0.65      0.60      2500
weighted avg       0.76      0.65      0.60      2500



In [311]:
from sklearn.svm import SVC
# TODO： 利用SVM来训练模型
#       1. 评估方式： F1-score
#       2. 超参数（hyperparater）的选择利用grid search https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html
#       3. 打印出在测试数据中的最好的结果（precision, recall, f1-score, 需要分别打印出正负样本，以及综合的）
#       请注意：做交叉验证时绝对不能用测试数据。 测试数据只能用来最后的”一次性“检验。
#       SVM的使用方法请参考：http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html
#       对于SVM模型，经常调整的超参数为：C, gamma, kernel。 这里的参数C跟逻辑回归是一样的， gamma和kernel是针对于SVM的参数
#       在这里先不要考虑他们的含义（或者通过官方文档试图理解一下）， 在课程最后的部分会讲到这些内容。 
params_c = np.logspace(-5,2,15)
params_gamma = [1, 0.1, 0.001, 0.0001]#['auto', 'scale', 'auto_deprecated']
params_kernel = ['linear', 'rbf']

clf = SVC()
parameters = {'C': params_c,
              'gamma': params_gamma,
              'kernel': params_kernel}

model = GridSearchCV(clf, parameters, scoring = 'f1', cv = 5)
model.fit(X_train, y_train)

# print confusion matrix
predictions = model.predict(X_test)
print(classification_report(y_test, predictions.astype(str)))

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='auto_deprecated', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='warn', n_jobs=None,
             param_grid={'C': array([1.00000000e-05, 3.16227766e-05, 1.00000000e-04, 3.16227766e-04,
       1.00000000e-03, 3.16227766e-03, 1.00000000e-02, 3.16227766e-02,
       1.00000000e-01, 3.16227766e-01, 1.00000000e+00, 3.16227766e+00,
       1.00000000e+01, 3.16227766e+01, 1.00000000e+02]),
                         'gamma': [1, 0.1, 0.001, 0.0001],
                         'kernel': ['linear', 'rbf']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='f1', verbose=0)

In [313]:
print(model.best_params_)
print(classification_report(y_test, predictions.astype(str)))

{'C': 1.0, 'gamma': 1, 'kernel': 'rbf'}
              precision    recall  f1-score   support

           0       0.94      0.31      0.47      1250
           1       0.59      0.98      0.73      1250

    accuracy                           0.65      2500
   macro avg       0.76      0.65      0.60      2500
weighted avg       0.76      0.65      0.60      2500



> 对于超参数的调整，我们经常使用gridsearch，这也是工业界最常用的方法，但它的缺点是需要大量的计算，所以近年来这方面的研究也成为了重点。 其中一个比较经典的成果为Bayesian Optimization（利用贝叶斯的思路去寻找最好的超参数）。Ryan P. Adams主导的Bayesian Optimization利用高斯过程作为后验概率（posteior distribution）来寻找最优解。 https://papers.nips.cc/paper/4522-practical-bayesian-optimization-of-machine-learning-algorithms.pdf 在下面的练习中，我们尝试使用Bayesian Optimization工具来去寻找最优的超参数。参考工具：https://github.com/fmfn/BayesianOptimization  感兴趣的朋友可以去研究一下。 

### 6. 思考题 
1. 对于情感分析来说，有一个问题也很重要，比如一个句子里出现了 “我不太兴奋”， “不是很满意”。 在这种情况，因为句子中出现了一些积极的词汇很可能被算法识别成正面的，但由于前面有一个“不”这种关键词，所以否定+肯定=否定，算法中这种情况也需要考虑。另外，否定+否定=肯定， 这种情况也一样。 
2. 另外一个问题是aspect-based sentiment analysis, 这个指的是做情感分析的时候，我们既想了解情感，也想了解特定的方面。 举个例子： “这部手机的电池性能不错，但摄像不够清晰啊!”, 分析完之后可以得到的结论是： “电池：正面， 摄像：负面”， 也就是针对于一个产品的每一个性能做判定，这种问题我们叫做aspect-based sentiment analysis，也是传统情感分析的延伸。

>``Q``: 对于如上两个问题，有什么解决方案？ 大概列一下能想到的处理方案。 用简介的文字来描述即可。  

// 你的答案在这里.......
1. Instead of using single word as token, use bigram/trigram or more words.
2. Instead of using binary sentiment label, use sentiment scores. For example, scores that runs between -5 and 5.

### 7. 其他领域（仅供参考）
跟情感分析类似的领域有叫affective computing, 也就是用来识别情绪(emotion recognition)。但情感和情绪又不太一样，情绪指的是高兴，低落，失落，兴奋这些人的情绪。我们知道真正的人工智能是需要读懂人类的情绪的。而且情绪识别有很多场景，比如服务机器人根据不同的情绪来跟用户交流； 无人驾驶里通过识别用户的情绪（摄像头或者声音或者传感器）来保证安全驾驶； IOT领域里设备也需要读懂我们的情绪； 微博里通过文本读懂每个人发文时的情绪。 

总体来讲，情绪识别跟情感识别所用到的技术是类似的，感兴趣的小伙伴，也可以关注一下这个领域。 如果想发论文，强烈建议选择情绪方面的，不建议选择情感分析，因为问题太老了。情绪分析是近几年才开始受关注的领域。 